## WPL Financial Model

### Next Steps 
- QC on more model inputs (Rowen gave me a new set of models to try out)
- Implement some QC functionality
    1. Logical checks (e.g., do the columns have the correct range of values? are numbers negative in places they shouldn't be?)
    2. Financial checks

### Features to Add
- Make a summary output of all runs that matches what we do currently in Excel
- Automate some of the input assumption calculations (i.e., the things that are not hard coded in Excel model's input sheets)
    - Only worth doing for data that usually comes in a specified format
- Potentially add toggles this workbook to switch between scenarios easily and see all the outputs? 
- Implement more QC functionality
- Add more explanations of how/why things are they way they are
    - Could potentially use this as a financial model training tool, in which we explain the calculations more than is possible in an Excel workbook


### Excel Model Differences


- In excel model like 353 (New CapEx for captial charge calculation), we take the cumulative sum of new CapEx every year. After the end-effects period started, we stopped adding in 2022 CapEx. In the Python, we add in the 2022 values into the end-effects period (and I updated a copy of the excel to match). Not clear if this is intentional.
- In excel model, the rate base sum of depreciation was not including CA1/2 solar depreciation during the end-effects period, so I added that back in (in the Python and copy of the model).
- In the excel model, line 54 (with FOM costs for Gas) was incorrectly index-matching to the input sheet, so all the FOM costs were off by a year.
- [*Did not change this but need to verify that this is intentional*] There is a change in several lines of the capital charge section starting in 2055. We basically switch from using normal formulas to the end-effects formulas that just inflate the previous year's value. 2055 is not the start of end-effects or solar extension, which is confusing because this formula change only happens at the start of one of those periods.

### Python Pros & Cons

*Pros:*
- Super easy to scale the model in Python. We can input any number of plants and the script will work. In excel, would need to reformat the entire workbook to get it to work. Extending formulas in excel can be a bit unstable if cells are not frozen correctly.
- Harder to make mistakes in some ways (e.g., You can't accidently move a cell in the wrong place)
- Way easier to add documentation (whether through comments or chunks of text like these)
- Easier to understand what information is getting processed (since we can name variables in formulas instead of having everything referencing cell numbers and letters)
- Consecutive logic is easier to follow (in my opinion)
- Easier to make and remove changes: you can leave the base revenue requirement calculations alone and make one-off changes in a more isolated way than is possible in Excel
- Faster running time (might not be a huge deal for WPL model - would be more relevant to something like the competitive transmission financial moderl that is a lot more dynamic)

*Cons:*
- Some loss of visibility: e.g., Can't quickly look into a formula to see how a number is calculated 
- Excel is easier for quick calculations that can be helpful during modeling
- More people are familiar with Excel, which is important for QC'ing
    - However, it may not be the case that people are actually diving into the formulas when they QC - should consider what we can do with the Python output to be as QC'able as possible
- Easier to make formatting look nice



### Import packages

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
import pathlib
from datetime import date
import numpy_financial as npf
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, numbers, Border, Side

import re

# Pause future warnings for cleaner output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Import Data and Supporting Functions

In [2]:
# Set the path to the folder containing your Excel files
folder_path = '/Users/alomsadze/OneDrive - Charles River Associates International/Desktop/WPL/Python Version/'
model_inputs = pd.ExcelFile(folder_path + "Direct Model Inputs.xlsx")

sys.path.append(folder_path + '/funcs/')

# Import functions we wrote to support calculations from separate .py files
from data_processing_functions import *
from O_and_M_functions import *
from plant_specific_functions import *
from depreciation_functions import *
from deferred_tax_functions import *
from tax_credit_functions import *
from capital_charge_functions import *
from excel_output_funcs import *

**Loading Individual Sheets & Preprocessing**

The 'Model Inputs' workbook is a single workbook that contains all data used for our model assumptions. 
- We "parse" each sheet individually to read it in. 
- Some sheets have multiple tables, so we use a custom function to save those tables in a dictionary. 
- We also remove trailing and leading white space from some of the tables' column names, as whitespaces might arise when tables are made in Excel, which can cause issues down the line while coding (such as indexing a dataframe by the name of a column).

In [3]:
# scalar financial inputs
financial_inputs = model_inputs.parse("Financial Inputs", header=None)
financial_inputs_tables = read_excel_with_tables(financial_inputs)
financial_scalars_inputs = financial_inputs_tables['Scalar Inputs'].set_index('Scalar Input')
# inflation vector
inflation_vector_df = financial_inputs_tables['Inflation Vector - Base Year 2021$']
inflation_vector = inflation_vector_df.set_index('Year')['Scalar']

# AURORA output files
aurora_portfolio_summary = model_inputs.parse("Portfolio Summary")
aurora_portfolio_resource = model_inputs.parse("Portfolio Resource")

# SCENARIO: Baseline - financials
baseline_scenario_financials = model_inputs.parse("Baseline", header=None)
baseline_scenario_financials_tables = read_excel_with_tables(baseline_scenario_financials)
# SCENARIO: Datacenter - financials
datacenter_scenario_financials = model_inputs.parse("Datacenter Scenario Financials", header=None)
datacenter_scenario_financials_tables = read_excel_with_tables(datacenter_scenario_financials)
# SCENARIO: Datacenter No Ext- financials
datacenter_no_ext_scenario_financials = model_inputs.parse("Datacenter No Ext Scenario", header=None)
datacenter_no_ext_scenario_financials_tables = read_excel_with_tables(datacenter_no_ext_scenario_financials)

# capacity payment info
capacity_payments = model_inputs.parse("Capacity Payments")
# CCS inputs
CCS_inputs = model_inputs.parse("CCS", header=None)
CCS_inputs_tables = read_excel_with_tables(CCS_inputs)
# hydrogen island inputs
hydrogen_island_inputs = read_excel_with_tables(model_inputs.parse("Hydrogen Island", header=None))
# ancillary revenue inputs
AS_RT_inputs = remove_whitespaces_from_df(model_inputs.parse("AS_RT Value"))
# Capital Costs inputs
capital_costs = remove_whitespaces_from_df(model_inputs.parse("Capital Costs"))
# PTC and ITC inputs
ptcs_and_itcs = model_inputs.parse("PTCs and ITCs", header=None)
ptcs_and_itcs_tables = read_excel_with_tables(ptcs_and_itcs)
# AGP inputs
AGP_inputs = remove_whitespaces_from_df(model_inputs.parse("AGP"))


## Revenue Requirement Calculations

Set global variables --- this section is in a temporary state. The ultimate code will run over all IDs/Scenarios, but for now, we're just using these to build the model.

In [94]:
# Scenario and Run Variables 
aurora_iteration = 'CIC'#'CIC'MS_AGP NR_AGP ACT_AGP AD_AGP
iteration = 'Continue_Change'#'Continue_Change' Market_Stagnation New_Regulation Advanced_Customers Accelerated_Decarbonization
year = 2023 
aurora_portfolio_ID = 2 #(1-baseline, 2- datacenter, 5-datacenter no ext) # IMPROVEMENT TO DO - LINK THIS TO CASE_NAME SO WE ONLY NEED TO CHANGE CASE NAME
aurora_condition = 'ATC'
use_IRA = True
case_name = 'Datacenter' #'Datacenter_no_ext' Datacenter Baseline

# Set current scenario financials tables
scenario_financials_tables = datacenter_scenario_financials_tables #datacenter_scenario_financials_tables

# Years we want to calculate revenue requirement for 
rev_req_start_year = 2023
rev_req_end_year = 2047
end_effects_end_year = 2057
solar_extension_end_year = 2072
end_effects = True
solar_extension = True
inflation_rate = 0.021

# Make a dict of these variables so we can easily pass them into functions using just one variable
run_variables_dict = {
    'case_name': case_name,
    'iteration': iteration,
    'aurora_iteration': aurora_iteration,
    'aurora_condition': aurora_condition,
    'use_IRA': use_IRA,
    'year': year,
    'aurora_portfolio_ID': aurora_portfolio_ID,
    'rev_req_start_year': rev_req_start_year,
    'rev_req_end_year': rev_req_end_year,
    'end_effects_end_year': end_effects_end_year,
    'solar_extension_end_year': solar_extension_end_year}

### 1. O&M Summary

Prior to calculations, reformat some of the input data to make it easier to handle in our calculations. Theortically, we will remove this once section once all the input data is caclulated in python instead of Excel

In [95]:
cumulative_installed_capacity_MW_df = scenario_financials_tables['Cumulative Installed Capacity (MW)']
cumulative_installed_capacity_MW_df = cumulative_installed_capacity_MW_df.set_index('Category')

FOM_2021_kw_year_df = scenario_financials_tables['Fixed O&M ($2021/kW-yr)']
FOM_2021_kw_year_df = FOM_2021_kw_year_df.set_index('Category')

AS_RT_curr_inputs = AS_RT_inputs[AS_RT_inputs.Scenario == iteration]
AS_RT_curr_inputs = AS_RT_curr_inputs.drop(columns = 'Scenario').set_index('Year').T

FOM_years = cumulative_installed_capacity_MW_df.columns.values

# capacity additions inputs
new_capacity_additions_annual_df = scenario_financials_tables['New Capacity Additions Annual (MW)'].set_index('Year')
new_capacity_additions_annual_df = new_capacity_additions_annual_df.T

####  Calculate Variable O&M , Fixed O&M, Subhourly & Ancillary Revenue from new builds


In [96]:
VOM_portfolio_cost_df = calc_VOM(run_variables_dict, 
                          aurora_portfolio_summary,
                          capacity_payments,
                          end_effects=True,
                          solar_extension=True)

FOM_yearly_general_df = calc_FOM(run_variables_dict,
                                 scenario_financials_tables, 
                                 FOM_years, 
                                 financial_scalars_inputs)

FOM_portfolio_cost_df = calc_new_resource_FOM(run_variables_dict,
                                              FOM_years,
                                              cumulative_installed_capacity_MW_df, 
                                              FOM_2021_kw_year_df, 
                                              inflation_vector,
                                              CCS_inputs_tables,
                                              hydrogen_island_inputs)

AS_RT_portfolio_cost_df = calc_new_resource_AS_RT(run_variables_dict,
                                                  FOM_years,
                                                  AS_RT_curr_inputs, 
                                                  cumulative_installed_capacity_MW_df)

In [97]:
VOM_portfolio_cost_df = calc_VOM(run_variables_dict, 
                          aurora_portfolio_summary,
                          capacity_payments,
                          end_effects=True,
                          solar_extension=True)
VOM_portfolio_cost_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072
Total Owned Cost,537460188,471145219,353665406,262398250,253670625,269277125,293761969,308243188,325586125,245003406,229553406,228581312,267606250,267590438,263890875,268308562,276112906,270473062,286528219,293996719,333906562,333948344,350943906,232417156,234832531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Market Purchases (Energy),27016125,16312913,15318784,37247598,57834707,64844215,63107496,53082539,48967758,86054789,90726930,100006852,42647191,49081227,54515609,60523109,64799777,82657305,91013797,120368688,116709039,133438297,130535000,197774578,175095375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Market Sales (Energy),-238602703,-222928328,-170422375,-82738203,-59791453,-63964172,-72880945,-75051695,-104529734,-57136230,-55638488,-50584398,-109046227,-101741062,-95773750,-93005797,-89264625,-77068508,-79082344,-69249039,-85151367,-78669516,-151876078,-92851477,-146520562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Contract Cost,39703293,56600320,61741785,55335277,51587680,52316910,53354586,54367176,55156062,56047465,55717469,56484090,50655504,41177262,41889672,42291520,42950086,43720934,40814289,12708388,12678766,4038387,4049065,4039085,4038613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Contract Sales,-5462529,-5586876,-5701747,-5798350,-5930664,-6021166,-1740299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
High Load Capacity Payment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Net Market Purchases,-211586578,-206615415,-155103591,-45490605,-1956746,880043,-9773449,-21969156,-55561977,28918559,35088441,49422453,-66399035,-52659836,-41258141,-32482688,-24464848,5588797,11931453,51119648,31557672,54768781,-21341078,104923102,28574812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Total Portfolio Cost,360114373,315543248,254601854,266444572,297370895,316452912,335602806,340641207,325180211,329969430,320359316,334487855,251862719,256107863,264522406,278117395,294598145,319782793,339273961,357824755,378143000,392755512,333651893,341379343,267445956,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Combine all VOM data

In [98]:
dfs_to_stack = [VOM_portfolio_cost_df, FOM_yearly_general_df, FOM_portfolio_cost_df, AS_RT_portfolio_cost_df]
O_and_M_summary = stack_dataframes(dfs_to_stack)

# calculate total O&M and put it into summary
total_O_and_M_costs = O_and_M_summary.loc[['Total Portfolio Cost', 
                                     'High Load Capacity Payment', 
                                     'FOM', 
                                     'Transmission Upgrade OpEx', 
                                     'DSM Costs', 
                                     'Tax Equity Costs - CA1 & CA2', 
                                     'New Unit FOM', 
                                     'New Unit Subhourly / Ancillary Revenue']].sum(axis=0, skipna=True)

# Insert total O&M cost row into first row of dataframe and format
O_and_M_summary = pd.concat([O_and_M_summary.iloc[:0], pd.DataFrame([total_O_and_M_costs]), O_and_M_summary.iloc[0:]])
O_and_M_summary.rename(index={O_and_M_summary.index[0]: 'Total O&M Costs'}, inplace=True)

O_and_M_summary.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072
Total O&M Costs,413693636,385554966,320583869,332180918,374898402,398425054,427956521,433187195,429466463,415350351,416142803,433348932,469677497,478973446,491737064,508651532,530623764,557650224,580343081,606928004,643818660,661952787,620560839,619340944,576329668,182951993,186793985,190716659,194721708,198810864,202985892,207248596,211600817,216044434,220581367,225213576,229943061,234771865,239702074,244735818,249875270,255122651,260480226,265950311,271535268,277237508,283059496,289003745,295072824,301269353,307596010
Total Owned Cost,537460188,471145219,353665406,262398250,253670625,269277125,293761969,308243188,325586125,245003406,229553406,228581312,267606250,267590438,263890875,268308562,276112906,270473062,286528219,293996719,333906562,333948344,350943906,232417156,234832531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Market Purchases (Energy),27016125,16312913,15318784,37247598,57834707,64844215,63107496,53082539,48967758,86054789,90726930,100006852,42647191,49081227,54515609,60523109,64799777,82657305,91013797,120368688,116709039,133438297,130535000,197774578,175095375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Market Sales (Energy),-238602703,-222928328,-170422375,-82738203,-59791453,-63964172,-72880945,-75051695,-104529734,-57136230,-55638488,-50584398,-109046227,-101741062,-95773750,-93005797,-89264625,-77068508,-79082344,-69249039,-85151367,-78669516,-151876078,-92851477,-146520562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Contract Cost,39703293,56600320,61741785,55335277,51587680,52316910,53354586,54367176,55156062,56047465,55717469,56484090,50655504,41177262,41889672,42291520,42950086,43720934,40814289,12708388,12678766,4038387,4049065,4039085,4038613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Contract Sales,-5462529,-5586876,-5701747,-5798350,-5930664,-6021166,-1740299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
High Load Capacity Payment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Net Market Purchases,-211586578,-206615415,-155103591,-45490605,-1956746,880043,-9773449,-21969156,-55561977,28918559,35088441,49422453,-66399035,-52659836,-41258141,-32482688,-24464848,5588797,11931453,51119648,31557672,54768781,-21341078,104923102,28574812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Total Portfolio Cost,360114373,315543248,254601854,266444572,297370895,316452912,335602806,340641207,325180211,329969430,320359316,334487855,251862719,256107863,264522406,278117395,294598145,319782793,339273961,357824755,378143000,392755512,333651893,341379343,267445956,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
FOM,50201215,59707422,56192857,58415754,71870574,68329971,70350836,62381446,71355625,43302290,44179983,45311958,45778741,47505085,48420670,48822430,51160695,51096582,51041855,51720035,52394896,52576138,52330266,33675491,34349001,26586154,27144463,27714497,28296501,28890728,29497433,30116879,30749333,31395069,32054366,32727507,33414785,34116496,34832942,35564434,36311287,37073824,37852374,38647274,39458867,40287503,41133541,41997345,42879289,43779754,44699129


### 2. Existing Plant

In [99]:
existing_plant_NPV_BOY, existing_plant_NPV_EOY, existing_plant_depreciation, total_existing_plant_summary = calc_existing_plant_summary(run_variables_dict, scenario_financials_tables, financial_scalars_inputs)
#total_existing_plant_summary.style.format(precision=0)

df2 is missing columns: 2058, 2068, 2048, 2049, 2066, 2067, 2050, 2061, 2059, 2064, 2063, 2053, 2056, 2052, 2051, 2054, 2060, 2071, 2062, 2055, 2065, 2070, 2072, 2057, 2069


### 3. Rate Base Inputs (CapEx, Ongoing CapEx)

#### New CapEx Calculations

In [100]:
# Prepare capital costs
curr_capital_costs = capital_costs[capital_costs.Scenario == iteration].drop(columns = 'Scenario')
curr_capital_costs = curr_capital_costs.set_index('Year').T

# Create New CapEx Table (calculates as capacity * 1000 * capital costs * inflation)
new_capex_df = convert_capacity_table_to_cost_table(new_capacity_additions_annual_df,
                                        curr_capital_costs, 
                                        inflation_vector,
                                        name_adjuster = 'New CapEx -')

# AGP specific adjustments requested by client
sheb_neenah_units = ['Neenah CT1', 'Neenah CT2', 'Sheboygan CT1', 'Sheboygan CT2']
AGP_sheb_neenah_inputs = AGP_inputs[AGP_inputs['Unit'].isin(sheb_neenah_units)]
AGP_sheb_neenah_inputs['Cap Costs ($2021)'].sum() * inflation_vector[2026]
new_capex_df.loc['New CapEx - AGP Neenah & Sheboygan', 2026] = AGP_sheb_neenah_inputs['Cap Costs ($2021)'].sum() * inflation_vector[2026] 

# The version of the new_capex_df with accumulated AFUDC is calculated in the AFUDC section because it needs those values
new_capex_df.style.format(precision=0)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,,,,,,,,,,,,,,,,,,,,,,,,,,
New CapEx - Wind - WI,0,0,0,0,0,349401676,342210760,334980111,327257431,319024420,310262208,0,0,0,0,0,0,0,0,0,330194133,0,340425398,0,354873392,0
New CapEx - H2 Island,0,0,0,0,0,0,0,0,0,0,0,0,0,124178589,0,0,0,0,0,0,0,0,0,0,0,0
New CapEx - H2 Enabled RICE,0,0,0,0,0,412534529,0,159675883,1083252511,0,0,0,0,0,0,0,0,63399572,322544454,65636842,0,136017815,1388741888,0,0,0
New CapEx - Paired Li Ion Battery_10% Bonus ITC,0,0,0,419104755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
New CapEx - Standalone Li Ion Battery,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1001445633,0,0,0
New CapEx - Paired Li Ion Battery,0,0,154321418,0,216661243,0,0,0,237286037,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
New CapEx - Paired Li Ion Battery_no ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
New CapEx - Solar and Storage - WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,845430937,258955496,881311872,0
New CapEx - CA1 - 2022,484674988,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Ongoing CapEx Calculations

In [101]:
# Handle extension period
def add_new_years(df, start_year, end_year, inflation_rate): 
    # Create a range of new years
    new_years = range(start_year, end_year + 1)
    # Create a new DataFrame with zeros
    new_data = pd.DataFrame(0, columns=new_years, index=df.index)
    
    # Update each row with cumulative inflation rates
    for row in df.index:
        last_value = df.loc[row].iloc[-1]
        for year in new_years:
            last_value *= (1 + inflation_rate)
            new_data.at[row, year] = last_value

    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df, new_data], axis=1)
    return result_df       

In [102]:
### Existing Resource Ongoing CapEx

ongoing_capex_by_plant_df = scenario_financials_tables['Ongoing CapEx by Plant Summary'].set_index('Category')
ongoing_capex_by_plant_df = ongoing_capex_by_plant_df.drop(['Decomissioning', 'FOM', 'DSM Costs'])
# we hard code ongoing capex for transmission upgrade cost as 0 
ongoing_capex_by_plant_df.loc['Transmission Upgrade OpEx'] = 0

# Account for extension period
if end_effects:
    end_year = run_variables_dict['end_effects_end_year']
    # add extension data
    ongoing_capex_by_plant_df = add_new_years(ongoing_capex_by_plant_df, 
                                            ongoing_capex_by_plant_df.columns[-1] + 1, 
                                            end_year, 
                                            inflation_rate)

### New Resource Ongoing CapEx

# Get Ongoing CapEx per resource & Capacity by resource
new_resource_ongoing_capex = scenario_financials_tables['Ongoing CapEx ($2021/kW-yr)'].set_index('Category')
new_resource_ongoing_capex = new_resource_ongoing_capex.drop('New4') # Drop New4 line

# Calculate total ongoing capex by resource
ongoing_capex_by_new_resource_df = convert_capacity_table_to_cost_table(cumulative_installed_capacity_MW_df, 
                                         new_resource_ongoing_capex, 
                                         inflation_vector, 
                                         name_adjuster = 'Ongoing CapEx -')

# Add total to datafgrame
total_new_resource_ongoing_capex = ongoing_capex_by_new_resource_df.sum(axis=0)
ongoing_capex_by_new_resource_df = pd.concat([ongoing_capex_by_new_resource_df.iloc[:0], pd.DataFrame([total_new_resource_ongoing_capex]), ongoing_capex_by_new_resource_df.iloc[0:]])
ongoing_capex_by_new_resource_df.rename(index={ongoing_capex_by_new_resource_df.index[0]: 'Ongoing CapEx - New - Total'}, inplace=True)

### Stack
ongoing_capex_df = stack_dataframes([ongoing_capex_by_plant_df, ongoing_capex_by_new_resource_df])
ongoing_capex_df.style.format(precision=0)

df2 is missing columns: 2049, 2053, 2056, 2048, 2057, 2052, 2051, 2054, 2021, 2050, 2055


,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
Ongoing CapEx - Columbia 1,6621600,4974210,1537500,1568250,533205,10000000,4000000,4000000,869375,869375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ongoing CapEx - Columbia 2,6621600,4974210,1537500,522750,2666025,10000000,7000000,1000000,1000000,1000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ongoing CapEx - Edgewater 5,1000000,500000,10250000,522750,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ongoing CapEx - Riverside,11670195,15642322,16473340,16802806,17138862,19143212,17831273,21444631,18551656,23666317,19299760,21594380,58260019,20475192,20884696,21302390,21728437,52700788,23011200,23058392,11759780,2398995,1223487,1247957,0,0,0,0,0,0,0,0,0,0,0,0,0
Ongoing CapEx - Other Gen,30791233,42021232,27014187,22285343,32878104,230652371,34056988,24122372,24604819,25096916,30063465,38400495,27147859,31203445,36632421,32464065,29586325,29405037,29993137,30593000,31204860,31828957,32465537,33114847,33777144,34452687,26691914,27252444,27824745,28409065,29005655,29614774,30236684,30871655,31519960,32181879,32857698
Transmission Upgrade OpEx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ongoing CapEx - New - Total,None,0,0,2026015,7428722,10380373,10385629,10390996,10396475,13778761,13784473,13790305,13796259,13802339,13808546,13814883,13821353,13827960,13834705,13841591,13848623,14139444,14436372,34625905,36820807,42589313,43483688,None,None,None,None,None,None,None,None,None,None
Ongoing CapEx - Wind - WI,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,None
Ongoing CapEx - H2 Island,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,None
Ongoing CapEx - H2 Enabled RICE,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,None


### 4. AFUDC Calculations

To - do: confirm the choice to start AFUDC schedule in 2022

In [103]:
# Create a version of the capex table to use for calcs
new_capex_df_copy = new_capex_df.copy()
new_capex_df_copy.index = new_capex_df_copy.index.str.replace('New CapEx - ', '')

# Create empty AFUDC schedule df
AFUDC_schedule_df = new_capex_df_copy.copy() * 0
AFUDC_schedule_df = AFUDC_schedule_df.drop(columns=2022) # dropping 2022 to match format in Excel # To - do: confirm this choice

# Get unit spend schedules
new_unit_spend_schedule_df = remove_whitespaces_from_df(financial_inputs_tables['New Unit Spend Schedule (% of total spend)'])
new_unit_spend_schedule_with_metadata_df = remove_whitespaces_from_df(financial_inputs_tables['New Unit Spend Schedule with Metadata'])
new_unit_spend_schedule_with_metadata_df = new_unit_spend_schedule_with_metadata_df.set_index('Year of Construction')


In [104]:
### Calculate AFUDC Schedule
AFUDC_schedule_df = new_capex_df_copy.copy() * 0
AFUDC_schedule_df = AFUDC_schedule_df.apply(lambda col: col.index.map(lambda row_index: calculate_AFUDC_schedule(col.name, row_index, new_capex_df_copy, new_unit_spend_schedule_df)))
#AFUDC_schedule_df.style.format(precision=0)

In [105]:
### Calculate AFUDC with Rate
AFUDC_with_rate_df = AFUDC_schedule_df.copy() * 0
AFUDC_with_rate_df = AFUDC_with_rate_df.apply(lambda col: col.index.map(lambda row_index: calculate_AFUDC_with_rate(col.name, row_index, new_capex_df_copy, new_unit_spend_schedule_with_metadata_df)))
#AFUDC_with_rate_df.style.format(precision=0)

In [106]:
### Calculate Accumulated AFUDC
AFUDC_accumulated_df = AFUDC_schedule_df.copy() * 0

for year in AFUDC_accumulated_df.columns:
    for plant in AFUDC_accumulated_df.index:
        capex = new_capex_df_copy.loc[plant, year]
        if capex != 0:
            AFUDC_rate_sum = sum(AFUDC_with_rate_df.loc[plant, AFUDC_with_rate_df.columns <= year])
            prev_AFUDC_accumulated_sum = sum(AFUDC_accumulated_df.loc[plant, AFUDC_with_rate_df.columns < year])
            AFUDC_accumulated_df.loc[plant, year] = AFUDC_rate_sum - prev_AFUDC_accumulated_sum

#AFUDC_accumulated_df

In [107]:
new_capex_with_accumulated_AFUDC = new_capex_df_copy.copy() + AFUDC_accumulated_df.copy()
#new_capex_with_accumulated_AFUDC.style.format(precision=0)

### 5. Depreciation Tables 

In [108]:
plant_book_and_tax_life = financial_inputs_tables['Book and Tax Life by Plant']
tax_depreciation_schedules = financial_inputs_tables['Tax Depreciation Schedules - Half Year Convention']
IRA_noIRA_depreciation_categories = financial_inputs_tables['Book and Tax Life - Depreciation Category']

# Get list of depreciation categories to make depeciaition tables for
depreciation_plants = list(plant_book_and_tax_life['Plant'])

In [109]:
# Make a book depreciation dictionary and tax depreciation dict where we can store depreciaiton tables

book_depreciation_tables_dict = {}
tax_depreciation_tables_dict = {}

# Set a fixed start year if needed, otherwise set as None
fixed_start_year = 2022

# Determine depreciaiton categories based on whether IRA is being modeled
if use_IRA == True:
    depreciation_categories = IRA_noIRA_depreciation_categories.drop(columns='With Tax Equity (no IRA)')
else:
    depreciation_categories = IRA_noIRA_depreciation_categories.drop(columns='Without Tax Equity (refundable through IRA)')
depreciation_categories = depreciation_categories.rename(columns={depreciation_categories.columns[0]: "Plant Type", 
                                                                  depreciation_categories.columns[1]: "Depreciation Category"})

# Reformat New CapEx DF that contains IRA/no-IRA dependent labeling
new_capex_by_depreciation_df = new_capex_with_accumulated_AFUDC.copy()
new_capex_by_depreciation_df['Plant Type'] = new_capex_by_depreciation_df.index.str.replace('New CapEx - ', '')
new_capex_by_depreciation_df = pd.merge(new_capex_by_depreciation_df, depreciation_categories, on='Plant Type', how='left')
new_capex_by_depreciation_df = new_capex_by_depreciation_df.drop(columns=['Plant Type'])
new_capex_by_depreciation_df = new_capex_by_depreciation_df.set_index('Depreciation Category')

# For "Other Gen" depreciation, we use the sum of ongoing capex of "Ongoing CapEx - Other Gen" and "New resources capex"
#   We update the ongoing capex table accordingly
ongoing_capex_df_adjusted = ongoing_capex_df.copy()
ongoing_capex_df_adjusted.loc['Ongoing CapEx - Other Gen'] = (ongoing_capex_df_adjusted.loc['Ongoing CapEx - Other Gen'] +
                                                     ongoing_capex_df_adjusted.loc['Ongoing CapEx - New - Total'].fillna(0))

# Loop over each plant to make depreciation tables for & add then to their dicts
for plant in depreciation_plants:
    
    # Find the depreciation schedule
    depreciation_schedule = plant_book_and_tax_life[plant_book_and_tax_life['Plant'] == plant]

    # Retrieve the ongoing or new CapEx for the plant
    no_capex = False
    if 'Ongoing' in plant:
        capex_stream = ongoing_capex_df_adjusted.loc[ongoing_capex_df_adjusted.index == plant]
    
    # Go to new CapEx table
    else:
        if plant not in new_capex_by_depreciation_df.index:
            # Return a zero row
            capex_stream = pd.DataFrame(0, index=[0], columns=new_capex_by_depreciation_df.columns)
        else:
            capex_stream = new_capex_by_depreciation_df[new_capex_by_depreciation_df.index == plant].groupby(level=0).sum()
    
    # If sum of capex is 0, there is no depreciation we can do
    sum_of_capex = capex_stream.sum(axis=1).values
    if sum_of_capex == 0:
        no_capex = True       
        
    # Create the depreciaiton tables and store them in their respective dictionaries
    if no_capex == True:
        book_depreciation_tables_dict[plant] = "None because no CapEx provided"
        tax_depreciation_tables_dict[plant] = "None because no CapEx provided"
    else:
        curr_book_depreciation = create_book_depreciation_schedule(capex_stream, int(depreciation_schedule['Book']), fixed_start_year)
        book_depreciation_tables_dict[plant] = curr_book_depreciation
        # if tax depreciation is 0, use the book depreciation as tax depreciation. otherwise, depreciate using a MACRS schedule
        if int(depreciation_schedule['Tax']) == 0:
            tax_depreciation_tables_dict[plant] = curr_book_depreciation
        else:
            tax_depreciation_tables_dict[plant] = create_tax_depreciation_schedule(capex_stream, int(depreciation_schedule['Tax']), tax_depreciation_schedules, fixed_start_year)

In [110]:
"""
# Display each dataframe in a nice format
for key, df in book_depreciation_tables_dict.items():
    display(key)
    if type(df) == pd.DataFrame:
        print('---> Book Depreciation table below:')
        display(df.style.format(precision=0))
    else:
        print('---> No depreciation because no CapEx provided for unit.')
 """

"\n# Display each dataframe in a nice format\nfor key, df in book_depreciation_tables_dict.items():\n    display(key)\n    if type(df) == pd.DataFrame:\n        print('---> Book Depreciation table below:')\n        display(df.style.format(precision=0))\n    else:\n        print('---> No depreciation because no CapEx provided for unit.')\n "

In [112]:
# Display each dataframe in a nice format
"""
# Display each dataframe in a nice format
for key, df in tax_depreciation_tables_dict.items():
    display(key)
    if type(df) == pd.DataFrame:
        print('---> Tax Depreciation table below:')
        display(df.style.format(precision=0))
    else:
        print('---> No depreciation because no CapEx provided for unit.')   
"""

"\n# Display each dataframe in a nice format\nfor key, df in tax_depreciation_tables_dict.items():\n    display(key)\n    if type(df) == pd.DataFrame:\n        print('---> Tax Depreciation table below:')\n        display(df.style.format(precision=0))\n    else:\n        print('---> No depreciation because no CapEx provided for unit.')   \n"

### 6. Deferred Tax Calcs


In [113]:
BOY_state_tax = scenario_financials_tables['Existing Capital - Tax Value - State - BOY'].set_index('Plant Name').loc[['Total']]
EOY_state_tax = scenario_financials_tables['Existing Capital - Tax Value - State - EOY'].set_index('Plant Name').loc[['Total']]

BOY_federal_tax = scenario_financials_tables['Existing Capital - Tax Value - Federal - BOY'].set_index('Plant Name').loc[['Total']]
EOY_federal_tax = scenario_financials_tables['Existing Capital - Tax Value - Federal - EOY'].set_index('Plant Name').loc[['Total']]


In [114]:
# Handle extension period
def add_new_years(df, start_year, end_year, inflation_rate): 
    # Create a range of new years
    new_years = range(start_year, end_year + 1)
    # Create a new DataFrame with zeros
    new_data = pd.DataFrame(0, columns=new_years, index=df.index)
    # Update the "New Unit FOM" row with cumulative inflation rates
    last_value = df.loc['Total'].iloc[-1]
    for year in new_years:
        last_value *= (1 + inflation_rate)
        new_data.at['Total', year] = last_value
    # Concatenate the existing DataFrame with the new DataFrame
    result_df = pd.concat([df, new_data], axis=1)
    return result_df    
    
if end_effects:
    end_year = run_variables_dict['end_effects_end_year']
if solar_extension:
    end_year = run_variables_dict['solar_extension_end_year']

if end_effects or solar_extension:
    BOY_state_tax = add_new_years(BOY_state_tax, BOY_state_tax.columns[-1]+1, end_year, inflation_rate)
    EOY_state_tax = add_new_years(EOY_state_tax, EOY_state_tax.columns[-1]+1, end_year, inflation_rate)
    BOY_federal_tax = add_new_years(BOY_federal_tax, BOY_federal_tax.columns[-1]+1, end_year, inflation_rate)
    EOY_federal_tax = add_new_years(EOY_federal_tax, EOY_federal_tax.columns[-1]+1, end_year, inflation_rate)


In [115]:
### State Deferred Taxes Only
state_tax_rate = financial_scalars_inputs.loc['State Income Tax Rate'][0]
deferred_tax_state_df = calc_deferred_taxes(state_tax_rate,
                                            BOY_state_tax, 
                                            EOY_state_tax,
                                            book_depreciation_tables_dict,
                                            tax_depreciation_tables_dict,
                                            existing_plant_depreciation, 
                                            total_existing_plant_summary, 
                                            existing_plant_NPV_BOY)
deferred_tax_state_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
Starting Deferred Tax Liability,0,1901654,8412000,17072862,32773904,54689676,79945981,109049636,139682893,174381100,213405037,247714144,274213839,297754868,316542647,324813224,325918364,324931761,320822295,314370722,312226918,315563404,318266718,350929888,415054657,479534558,537014581,568753432,573952066,561868442,538513961,507564914,479432087,451454589,423645367,395662582,369539521,344063633,316176962,289326754,263268225,238015665,212875132,191874993,171167683,150379665,129485041,108570767,87944498,68291900,49583341,31863062,14481732,7393047,1877418,1457981,1017514,None,None,None
Deferred Tax - New Capital,1901654,6510345,8660863,15701041,21915772,25256305,29103655,30633258,34698206,39023937,34309107,26499694,23541030,18787779,8270577,1105140,-986603,-4109466,-6451574,-2143803,3336485,2703315,32663170,64124769,64479901,57480022,31738851,5198634,-12083624,-23354481,-30949047,-28132827,-27977498,-27809222,-27982785,-26123060,-25475888,-27886671,-26850208,-26058528,-25252560,-25140533,-21000139,-20707310,-20788018,-20894625,-20914273,-20626269,-19652598,-18708559,-17720279,-17381330,-7088684,-5515629,-419437,-440467,None,None,None,None
Ending Deferred Tax Liability,1901654,8412000,17072862,32773904,54689676,79945981,109049636,139682893,174381100,213405037,247714144,274213839,297754868,316542647,324813224,325918364,324931761,320822295,314370722,312226918,315563404,318266718,350929888,415054657,479534558,537014581,568753432,573952066,561868442,538513961,507564914,479432087,451454589,423645367,395662582,369539521,344063633,316176962,289326754,263268225,238015665,212875132,191874993,171167683,150379665,129485041,108570767,87944498,68291900,49583341,31863062,14481732,7393047,1877418,1457981,1017514,None,None,None,0
Book Depreciation - New Capital,18880312,58551113,65444277,116515112,171063782,199392611,213237956,231943382,289446370,302606425,315899906,319868071,374958508,381950821,384654075,387259519,391096871,395883751,409334953,421752259,434693426,441152369,539730188,528880206,596709898,596792443,595610040,594973913,593684479,583637667,565735469,527108317,520982243,505696428,494950726,467915753,452540562,433432790,375653231,362315830,349368312,345530960,290386200,283547195,281274664,279339968,277414953,272542085,258966685,245740229,231926734,226302709,94696570,73399544,7491306,6306715,5097248,3862381,2601583,1314308
Tax Depreciation - New Capital,42943783,140932809,175038550,315195571,448385241,518985070,581514645,619575620,728516333,796414043,750046222,655194334,672845926,619690757,489309684,401243922,378612433,343882703,327696952,394624643,476913201,475360024,953048865,1340312927,1412636448,1324142755,997232421,660757258,440778698,288110830,174107250,171116443,166955900,153799449,140857486,137355404,130169507,80555770,35891575,32572067,29823244,27403485,24651130,21517577,18223767,14940075,12766424,11537956,10283354,9002741,7694900,6359930,4996589,3604954,2183760,733055,None,None,None,None
Net (T Less B) - New Capital,24063471,82381696,109594273,198680460,277321459,319592459,368276689,387632238,439069963,493807618,434146317,335326262,297887418,237739936,104655609,13984403,-12484438,-52001048,-81638001,-27127617,42219775,34207655,413318678,811432721,815926550,727350312,401622381,65783345,-152905781,-295526837,-391628220,-355991873,-354026342,-351896979,-354093239,-330560349,-322371055,-352877020,-339761656,-329743763,-319545067,-318127475,-265735071,-262029618,-263050898,-264399893,-264648528,-261004129,-248683331,-236737488,-224231834,-219942779,-89699980,-69794590,-5307546,-5573660,None,None,None,None
Cumulative Deferred Income Taxes - New Capital,1901654,8412000,17072862,32773904,54689676,79945981,109049636,139682893,174381100,213405

In [116]:
### Federal Deferred Taxes Only
federal_tax_rate = financial_scalars_inputs.loc['Federal Income Tax Rate'][0]

deferred_tax_federal_df = calc_deferred_taxes(federal_tax_rate,
                                            BOY_federal_tax, 
                                            EOY_federal_tax,
                                            book_depreciation_tables_dict,
                                            tax_depreciation_tables_dict,
                                            existing_plant_depreciation, 
                                            total_existing_plant_summary, 
                                            existing_plant_NPV_BOY)
deferred_tax_federal_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
Starting Deferred Tax Liability,0,5053329,22353485,45368282,87091179,145328685,212443102,289781206,371183976,463388669,567088268,658258995,728677510,791233868,841159254,863136932,866073657,863451925,852531705,835387725,829690925,838557078,845740686,932537608,1102938479,1274283055,1427026620,1511367320,1525181823,1493071609,1431010973,1348769047,1274010753,1199665222,1125766856,1051407276,981989602,914291681,840187507,768837559,699591369,632486905,565680135,509875770,454849550,399608862,344084884,288508693,233697826,181474327,131759454,84670769,38482786,19645790,4988926,3874341,2703873,None,None,None
Deferred Tax - New Capital,5053329,17300156,23014797,41722897,58237506,67114416,77338105,81402770,92204692,103699600,91170727,70418515,62556358,49925387,21977678,2936725,-2621732,-10920220,-17143980,-5696799,8866153,7183608,86796922,170400871,171344575,152743566,84340700,13814502,-32110214,-62060636,-82241926,-74758293,-74345532,-73898366,-74359580,-69417673,-67697921,-74104174,-71349948,-69246190,-67104464,-66806770,-55804365,-55026220,-55240688,-55523978,-55576191,-54810867,-52223500,-49714872,-47088685,-46187984,-18836996,-14656864,-1114585,-1170469,None,None,None,None
Ending Deferred Tax Liability,5053329,22353485,45368282,87091179,145328685,212443102,289781206,371183976,463388669,567088268,658258995,728677510,791233868,841159254,863136932,866073657,863451925,852531705,835387725,829690925,838557078,845740686,932537608,1102938479,1274283055,1427026620,1511367320,1525181823,1493071609,1431010973,1348769047,1274010753,1199665222,1125766856,1051407276,981989602,914291681,840187507,768837559,699591369,632486905,565680135,509875770,454849550,399608862,344084884,288508693,233697826,181474327,131759454,84670769,38482786,19645790,4988926,3874341,2703873,None,None,None,0
Book Depreciation - New Capital,18880312,58551113,65444277,116515112,171063782,199392611,213237956,231943382,289446370,302606425,315899906,319868071,374958508,381950821,384654075,387259519,391096871,395883751,409334953,421752259,434693426,441152369,539730188,528880206,596709898,596792443,595610040,594973913,593684479,583637667,565735469,527108317,520982243,505696428,494950726,467915753,452540562,433432790,375653231,362315830,349368312,345530960,290386200,283547195,281274664,279339968,277414953,272542085,258966685,245740229,231926734,226302709,94696570,73399544,7491306,6306715,5097248,3862381,2601583,1314308
Tax Depreciation - New Capital,42943783,140932809,175038550,315195571,448385241,518985070,581514645,619575620,728516333,796414043,750046222,655194334,672845926,619690757,489309684,401243922,378612433,343882703,327696952,394624643,476913201,475360024,953048865,1340312927,1412636448,1324142755,997232421,660757258,440778698,288110830,174107250,171116443,166955900,153799449,140857486,137355404,130169507,80555770,35891575,32572067,29823244,27403485,24651130,21517577,18223767,14940075,12766424,11537956,10283354,9002741,7694900,6359930,4996589,3604954,2183760,733055,None,None,None,None
Net (T Less B) - New Capital,24063471,82381696,109594273,198680460,277321459,319592459,368276689,387632238,439069963,493807618,434146317,335326262,297887418,237739936,104655609,13984403,-12484438,-52001048,-81638001,-27127617,42219775,34207655,413318678,811432721,815926550,727350312,401622381,65783345,-152905781,-295526837,-391628220,-355991873,-354026342,-351896979,-354093239,-330560349,-322371055,-352877020,-339761656,-329743763,-319545067,-318127475,-265735071,-262029618,-263050898,-264399893,-264648528,-261004129,-248683331,-236737488,-224231834,-219942779,-89699980,-69794590,-5307546,-5573660,None,None,None,None
Cumulative Deferred Income Taxes - New Capital,5053329,22353485,45368282,87091179,

In [117]:
### Blended State and Federal Deferred Taxes
deferred_tax_blended_df = deferred_tax_federal_df.copy()
blended_deferred_tax_new_capital = deferred_tax_federal_df.loc[['Deferred Tax - New Capital']] + deferred_tax_state_df.loc[['Deferred Tax - New Capital']]*(1 - federal_tax_rate)
blended_deferred_tax_existing_capital = deferred_tax_federal_df.loc[['Deferred Tax - Existing Capital']] + deferred_tax_state_df.loc[['Deferred Tax - Existing Capital']]*(1 - federal_tax_rate)
blended_deferred_tax_liability = deferred_tax_federal_df.loc[['Deferred Tax Liability - Existing']] + deferred_tax_state_df.loc[['Deferred Tax Liability - Existing']]*(1 - federal_tax_rate)

deferred_tax_blended_df = calc_deferred_taxes(federal_tax_rate,
                        BOY_federal_tax, 
                        EOY_federal_tax,
                        book_depreciation_tables_dict,
                        tax_depreciation_tables_dict,
                        existing_plant_depreciation, 
                        total_existing_plant_summary, 
                        existing_plant_NPV_BOY,
                        blended_tax_rate = True, 
                        blended_deferred_tax_new_capital = blended_deferred_tax_new_capital,
                        blended_deferred_tax_existing_capital = blended_deferred_tax_existing_capital,                      
                        blended_deferred_tax_liability = blended_deferred_tax_liability)

deferred_tax_blended_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
Starting Deferred Tax Liability,0,6555636,28998965,58855844,112982563,188533529,275600427,375930419,481533462,601149738,735678248,853953169,945306443,1026460214,1091227946,1119739379,1123549165,1120148017,1105981318,1083740595,1076350191,1087852167,1097171393,1209772220,1430831659,1653115356,1851268139,1960682532,1978603955,1936947678,1856437002,1749745329,1652762102,1556314346,1460446696,1363980715,1273925824,1186101951,1089967307,997405694,907573267,820519280,733851489,661457015,590072020,518408797,446378066,374279599,303173980,235424928,170930294,109842588,49923354,25486297,6472086,5026146,3507708,None,None,None
Deferred Tax - New Capital,6555636,22443329,29856879,54126719,75550966,87066898,100329992,105603044,119616275,134528510,118274921,91353274,81153771,64767732,28511434,3809785,-3401148,-14166698,-22240723,-7390404,11501976,9319226,112600826,221059439,222283697,198152783,109414393,17921423,-41656277,-80510676,-106691673,-96983227,-96447755,-95867651,-96465980,-90054891,-87823873,-96134644,-92561612,-89832428,-87053987,-86667791,-72394475,-71384995,-71663223,-72030731,-72098467,-71105620,-67749052,-64494634,-61087706,-59919234,-24437057,-19014211,-1445940,-1518438,None,None,None,None
Ending Deferred Tax Liability,6555636,28998965,58855844,112982563,188533529,275600427,375930419,481533462,601149738,735678248,853953169,945306443,1026460214,1091227946,1119739379,1123549165,1120148017,1105981318,1083740595,1076350191,1087852167,1097171393,1209772220,1430831659,1653115356,1851268139,1960682532,1978603955,1936947678,1856437002,1749745329,1652762102,1556314346,1460446696,1363980715,1273925824,1186101951,1089967307,997405694,907573267,820519280,733851489,661457015,590072020,518408797,446378066,374279599,303173980,235424928,170930294,109842588,49923354,25486297,6472086,5026146,3507708,None,None,None,0
Book Depreciation - New Capital,18880312,58551113,65444277,116515112,171063782,199392611,213237956,231943382,289446370,302606425,315899906,319868071,374958508,381950821,384654075,387259519,391096871,395883751,409334953,421752259,434693426,441152369,539730188,528880206,596709898,596792443,595610040,594973913,593684479,583637667,565735469,527108317,520982243,505696428,494950726,467915753,452540562,433432790,375653231,362315830,349368312,345530960,290386200,283547195,281274664,279339968,277414953,272542085,258966685,245740229,231926734,226302709,94696570,73399544,7491306,6306715,5097248,3862381,2601583,1314308
Tax Depreciation - New Capital,42943783,140932809,175038550,315195571,448385241,518985070,581514645,619575620,728516333,796414043,750046222,655194334,672845926,619690757,489309684,401243922,378612433,343882703,327696952,394624643,476913201,475360024,953048865,1340312927,1412636448,1324142755,997232421,660757258,440778698,288110830,174107250,171116443,166955900,153799449,140857486,137355404,130169507,80555770,35891575,32572067,29823244,27403485,24651130,21517577,18223767,14940075,12766424,11537956,10283354,9002741,7694900,6359930,4996589,3604954,2183760,733055,None,None,None,None
Net (T Less B) - New Capital,24063471,82381696,109594273,198680460,277321459,319592459,368276689,387632238,439069963,493807618,434146317,335326262,297887418,237739936,104655609,13984403,-12484438,-52001048,-81638001,-27127617,42219775,34207655,413318678,811432721,815926550,727350312,401622381,65783345,-152905781,-295526837,-391628220,-355991873,-354026342,-351896979,-354093239,-330560349,-322371055,-352877020,-339761656,-329743763,-319545067,-318127475,-265735071,-262029618,-263050898,-264399893,-264648528,-261004129,-248683331,-236737488,-224231834,-219942779,-89699980,-69794590,-5307546,-5573660,None,None,None,None
Cumulative Deferred Income Taxes - New Cap

### 7. Tax Credit Calculation & Normalized ITC


In [118]:
PTC_df = calculate_ptc(inflation_vector, financial_inputs_tables)
PTC_df.style.format(precision=0)  

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
PTC Price Kossuth,None,23,25,27,27,28,29,29,30,30,31,32,32,33,34,35,35,36,37,37,38,39,40,41,42,42,43,44,45
PTC Price,None,None,25,25,25,25,25,28,28,28,28,28,30,30,30,30,32,32,32,32,35,35,35,38,38,38,38,40,None
H2 PTC,None,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5
45Q Tax Credit,None,None,35,38,41,44,47,50,51,52,53,54,55,57,58,59,60,62,63,64,66,67,68,70,71,73,74,76,None


In [119]:
generation_df = calculate_generation(ptcs_and_itcs_tables, aurora_portfolio_resource, aurora_condition, aurora_iteration,
                                          aurora_portfolio_ID, hydrogen_island_inputs, cumulative_installed_capacity_MW_df,
                                          iteration, CCS_inputs_tables)
generation_df.style.format(precision=0)  


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Qualifying New Wind,0,0,0,0,576000,1277409,1983561,2673738,3363464,4063923,4807095,4788113,4787951,4772475,4197086,3499328,2814671,2108947,1403179,696245,0,0,0,0,0,0
Qualifying New Solar (Post-CA1/CA2),0,0,0,0,0,0,0,0,0,0,0,0,422157,419040,416953,414424,411458,409697,409859,405891,403803,402139,0,0,0,0
CA1 Solar,64387,732051,1461221,1447863,1437858,1430041,1426844,1418246,1414178,1404993,1396782,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CA2 Solar,0,220521,763166,757347,755395,751131,750461,743435,739181,734933,733352,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Kossuth,618567,619946,622061,618206,616535,617063,620812,622992,620323,618206,619561,618567,619946,622992,622225,616535,617063,618567,623543,620323,618206,616535,620479,619946,622992,0
Hydrogen,0,0,0,0,0,0,0,0,0,0,0,0,0,1642389,1642389,1642389,1642389,1642389,1642389,1642389,1642389,1642389,1642389,0,0,0
Gas CCGT with CCS,0,0,0,0,0,0,0,0,0,0,0,20589,1064078,1064078,1067270,1064078,1062535,1050564,1053437,1041586,1046654,1045457,1047984,0,0,0


In [120]:
old_tax_policy_PTC_generated = calculate_old_tax_policy_PTC_generated(PTC_df, generation_df, financial_inputs_tables, 
                                                                      use_IRA, financial_scalars_inputs)
old_tax_policy_PTC_generated.style.format(precision=0)

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Kossuth,0,0,0,16701302,16738528,17417704,17927961,17879502,18511894,18624364,19312752,14887755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Qualifying New Wind,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Captured CO2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194189,62780598,63844676,66170746,67036909,68002240,69337249,70580304,70827876,73265781,74227444,76502839,0,0,0
Grossed Up PTC,0,0,0,22954940,23006105,23939592,24640909,24574305,25443489,25598073,26544221,20462328,0,0,1641342,86288172,87750683,90947727,92138217,93465007,95299897,97008401,97348674,100699429,102021177,105148570,0,0,0


In [121]:
IRA_PTC_df = calculate_ira_ptc(generation_df, PTC_df, financial_scalars_inputs, use_IRA)
IRA_PTC_df.style.format(precision=0)  

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Qualifying New Wind,0,0,0,0,15840000,35128749,54547915,73527790,92495246,121917699,144212843,143643392,143638530,155105431,136405279,113728154,91476808,73813149,49111278,24368588,0,0,0,0,0,0
Qualifying New Solar (Post-CA1/CA2),0,0,0,0,0,0,0,0,0,0,0,0,12664709,13618809,13550969,13468766,13372375,14339409,14345055,14206186,15142627,15080227,0,0,0,0
CA1 Solar,1609664,18301285,36530536,36196575,39541090,39326125,39238200,39001772,38889896,42149794,41903445,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CA2 Solar,0,5513026,19079161,18933680,20773372,20656101,20637667,20444475,20327485,22047983,22000565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Hydrogen,0,0,0,0,0,0,0,0,0,0,0,0,0,6569556,6569556,6569556,6569556,8211945,8211945,8211945,8211945,8211945,8211945,0,0,0
Grossed Up PTC,2212387,32731344,76432199,75773235,104669746,130724340,157268637,182764850,208519921,255804577,286044151,197429240,214829442,240930825,215135344,183854010,153138383,132447239,98503756,64305543,32099459,32013694,11286826,0,0,0


In [122]:
### Total Grossed Up PTC Payment
if use_IRA == True:
    total_grossed_up_ptc = old_tax_policy_PTC_generated.loc[['Grossed Up PTC']] + IRA_PTC_df.loc[['Grossed Up PTC']] 
else:
    total_grossed_up_ptc = old_tax_policy_PTC_generated.loc[['Grossed Up PTC']]
total_grossed_up_ptc.style.format(precision=0)  

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Grossed Up PTC,nan,nan,nan,25167327,55737449,100371791,100414144,129244051,156167829,182866709,209309071,228982248,255804577,286044151,199070581,301117613,328681508,306083071,275992227,246603390,227747136,195512157,161654218,132798888,134034871,116435396,0,0,0


In [123]:
### NOL
NOL = financial_inputs_tables['Alliant Projected NOL?'].set_index('Year').T
NOL

Year,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
0,,,,,,,,,,,,,,,,,,,,,
Alliant Projected NOL?,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No


In [124]:
# Extract Storage ITC and update Total IRA ITC Benefit
storage_ITC = ptcs_and_itcs_tables['Storage ITC']
storage_ITC = storage_ITC[storage_ITC.Portfolio == run_variables_dict['case_name']]
storage_ITC.style.format(precision=0)  

0,Portfolio,Iteration,Year,2022.000000,2023.000000,2024.000000,2025.000000,2026.000000,2027.000000,2028.000000,2029.000000,2030.000000,2031.000000,2032.000000,2033.000000,2034.000000,2035.000000,2036.000000,2037.000000
1,Datacenter,Continue_Change,TOTAL ITC,0,0,8886281,41063992,57487341,57487341,48601061,16423350,13663627,13663627,13663627,13663627,0,0,0,0
2,Datacenter,Continue_Change_IRA,TOTAL ITC,0,0,0,25039828,36811031,36811031,36811031,11771203,9469034,9469034,9469034,9469034,0,0,0,0
3,Datacenter,Market_Stagnation,TOTAL ITC,0,0,0,32177711,48601061,48601061,48601061,16423350,13663627,13663627,13663627,13663627,0,0,0,0
4,Datacenter,New_Regulation,TOTAL ITC,0,0,0,34356020,51722868,51722868,51722868,17366848,13085536,13085536,13085536,13085536,0,0,0,0
5,Datacenter,Advanced_Customers,TOTAL ITC,0,0,0,34356020,51722868,51722868,51722868,17366848,13085536,13085536,13085536,13085536,0,0,0,0
6,Datacenter,Accelerated_Decarbonization,TOTAL ITC,0,0,0,34356020,51722868,51722868,51722868,17366848,13085536,13085536,13085536,13085536,0,0,0,0


In [125]:
ITC = calculate_ITC(NOL, financial_inputs_tables, financial_scalars_inputs, ptcs_and_itcs_tables, run_variables_dict)
ITC.style.format(precision=0)  

Year,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ITC %,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ITC Generated,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Accumulated Deferred ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Deferred ITC Asset,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Monetized ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Normalized ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Grossed Up ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Accumulated ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Accumulated Normalized ITC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 8. Rate Base Calculations


In [126]:
# Depreciation - new
new_depreciation = sum_annual_depreciation(book_depreciation_tables_dict)
new_depreciation = new_depreciation.rename(index={new_depreciation.index[0]: 'Depreciation - New'})

# Change in Deferred Tax Liability
changed_in_deferred_tax_liability = (deferred_tax_blended_df.loc['Deferred Tax - New Capital'] 
    + deferred_tax_blended_df.loc['Deferred Tax - Existing Capital'].fillna(0) 
    + ITC.loc['Change in Net Deferred Tax - ITC'])
changed_in_deferred_tax_liability = changed_in_deferred_tax_liability.to_frame().T
changed_in_deferred_tax_liability = changed_in_deferred_tax_liability.rename(index={changed_in_deferred_tax_liability.index[0]: 'Change in Deferred Tax Liability'})

# Depreciation Existing
depreciation_existing = existing_plant_depreciation.loc[['Total Depreciation']].copy()
depreciation_existing = depreciation_existing.rename(index={depreciation_existing.index[0]: 'Depreciation - Existing'})

# Additions to Existing Book
additions_to_existing_book = total_existing_plant_summary.loc[['Additions to Existing Book']].copy()

# CapEx
new_capex_with_accumulated_AFUDC_sum = new_capex_with_accumulated_AFUDC.sum()
new_capex_with_accumulated_AFUDC_sum = new_capex_with_accumulated_AFUDC_sum.reindex(ongoing_capex_df.columns, fill_value=0)
total_capex = new_capex_with_accumulated_AFUDC_sum + ongoing_capex_df.drop('Ongoing CapEx - New - Total').sum()
total_capex = total_capex.to_frame().T
total_capex = total_capex.rename(index={total_capex.index[0]: 'CapEx'})

In [127]:
# Make DF and initialize rate base values
dfs_to_stack = [total_capex, new_depreciation, changed_in_deferred_tax_liability, depreciation_existing, additions_to_existing_book]
rate_base_df = stack_dataframes(dfs_to_stack, print_warnings=False)

rate_base_df.loc['Starting Rate Base'] = 0
rate_base_df.loc['Ending Rate Base'] = 0

rate_base_df = rate_base_df.fillna(0)

# make sure the start year of the rate base is the year prior to the revenue requirement start year
rate_base_df = rate_base_df.loc[:, rate_base_df.columns.map(int) >= 2022]

In [128]:
# Initialize starting rate base
rate_base_start_year = min(rate_base_df.columns)
rate_base_df.loc['Starting Rate Base', rate_base_start_year] = (existing_plant_NPV_BOY.loc['Total NPV BOY', rate_base_start_year] 
    - deferred_tax_blended_df.loc['Deferred Tax Liability - Existing', rate_base_start_year])

# Fill in Starting and Ending rate base values
for year in rate_base_df.columns[1:]:
    
    # Starting Rate Base + CapEx, less depreciation and change in deferred taxes - max of 0 so it can never be negative
    rate_base_df.loc['Ending Rate Base', year - 1] = max(rate_base_df.loc['Starting Rate Base', year - 1] 
    + rate_base_df.loc['CapEx', year - 1] 
    - rate_base_df.loc['Depreciation - New', year - 1] 
    - rate_base_df.loc['Change in Deferred Tax Liability', year - 1]     
    - rate_base_df.loc['Depreciation - Existing', year - 1] 
    + rate_base_df.loc['Additions to Existing Book', year - 1], 0)
    
    # Update starting rate base
    rate_base_df.loc['Starting Rate Base', year] = rate_base_df.loc['Ending Rate Base', year - 1]


In [129]:
rate_base_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
CapEx,552786963,1178761527,198049332,1138572386,1208244720,835210095,403168759,550078320,1712207348,382172118,384047389,99204138,1639616921,195504251,67581337,65136116,95933784,130238614,390037437,361156551,378561529,184143211,3983783195,675417840,2019482354,70175602,27252444,27824745,28409065,29005655,29614774,30236684,30871655,31519960,32181879,32857698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Depreciation - New,18880312,58551113,65444277,116515112,171063782,199392611,213237956,231943382,289446370,302606425,315899906,319868071,374958508,381950821,384654075,387259519,391096871,395883751,409334953,421752259,434693426,441152369,539730188,528880206,596709898,596792443,595610040,594973913,593684479,583637667,565735469,527108317,520982243,505696428,494950726,467915753,452540562,433432790,375653231,362315830,349368312,345530960,290386200,283547195,281274664,279339968,277414953,272542085,258966685,245740229,231926734,226302709,94696570,73399544,7491306,6306715,5097248,3862381,2601583,1314308
Change in Deferred Tax Liability,927607,15745652,19201986,42959687,62512084,73789367,87050439,92316370,106336722,121241836,102098473,60445787,50246458,34490570,1379930,-23059549,-29112381,-34272671,-46315706,-31799794,-8728586,-10368198,93361064,210280584,215788270,191657356,102782561,11150324,-48569570,-87569148,-113898373,-104341267,-103960315,-103537974,-104297380,-98050750,-95987646,-104469856,-101071863,-98521394,-95925421,-95725525,-81642422,-80827148,-81303662,-81873619,-82148056,-81366250,-78225156,-75190736,-72008426,-59919234,-24437057,-19014211,-1445940,-1518438,0,0,0,0
Depreciation - Existing,150531723,146451255,142370787,142370787,136442037,136442037,136442037,136442037,136442037,136442037,136442037,134153860,134153860,131840156,120294171,119331197,113968878,89598425,89598425,89598425,74259394,72265724,70622511,39565448,23842466,23842466,24343158,24854364,25376305,25909208,26453301,27008821,27576006,28155102,28746359,29350033,29966383,30595677,31238187,31894189,32563966,33247810,33946014,34658880,35386717,36129838,36888564,37663224,38454152,39261689,40086184,0,0,0,0,0,0,0,0,0
Additions to Existing Book,0,-105682792,-101602324,0,0,-0,-0,-0,-0,0,-0,0,-0,0,0,-0,0,-0,-0,0,0,0,-0,-0,0,0,25022349,25547819,26084323,26632094,27191368,27762386,28345396,28940650,29548403,30168920,30802467,31449319,32109755,32784060,33472525,34175448,34893132,35625888,36374032,37137886,37917782,38714055,39527050,40357119,41204618,0,0,0,0,0,0,0,0,0
Starting Rate Base,2619956189,3002403509,3854734225,3724164184,4560890984,5399117802,5824703882,5791142209,5880518740,7060500960,6882382780,6711989753,6296726173,7376984268,7024206972,6585460134,6167065082,5787045498,5466074607,5403494372,5285100033,5163437328,4844530643,8124600075,8021291677,9204433397,8462316735,7791855769,7214249733,6698251906,6231911928,5810427672,5418650872,5033269990,4663417043,4305747621,3969559203,3613842371,3285733078,3012023279,2749118714,2496584381,2247706584,2039909924,1838156886,1639173198,1442714898,1248477219,1058352215,878683584,709229520,550429645,384046171,313786658,259401325,253355959,248567682,243470434,239608053,237006470
Ending Rate Base,3002403509,3854734225,3724164184,4560890984,5399117802,5824703882,5791142209,5880518740,7060500960,6882382780,6711989753,6296726173,7376984268,7024206972,6585460134,6167065082,5787045498,5466074607,5403494372,5285100033,5163437328,4844530643,8124600075,8021291677,9204433397,8462316735,7791855769,7214249733,6698251906,6231911928,5810427672,5418650872,5033269990,4663417043,4305747621,3969559203,3613842371,3285733078,3012023279,2749118714,2496584381,2247706584,2039909924,1838156886,1639173198,1442714898,1248477219,1058352215,878683584,709229520,5

### 9. Capital Charge Calculations


In [130]:
capital_charge_df = calculate_capital_charge(financial_scalars_inputs, 
                                             rate_base_df,
                                             end_effects=True,
                                             solar_extension=True,
                                             inflation_rate=0.021)
capital_charge_df.style.format(precision=0)

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
Ending Rate Base,3002403509,3854734225,3724164184,4560890984,5399117802,5824703882,5791142209,5880518740,7060500960,6882382780,6711989753,6296726173,7376984268,7024206972,6585460134,6167065082,5787045498,5466074607,5403494372,5285100033,5163437328,4844530643,8124600075,8021291677,9204433397,8462316735,7791855769,7214249733,6698251906,6231911928,5810427672,5418650872,5033269990,4663417043,4305747621,3969559203,3613842371,3285733078,3012023279,2749118714,2496584381,2247706584,2039909924,1838156886,1639173198,1442714898,1248477219,1058352215,878683584,709229520,550429645,384046171,313786658,259401325,253355959,248567682,243470434,239608053,237006470,0
Starting Equity ($),1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408,1321337408
Starting Debt ($),1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129,1119707129
New CapEx ($),552786963,1731548489,1929597821,3068170208,4276414928,5111625023,5514793782,6064872102,7777079450,8159251569,8543298957,8642503095,10282120017,10477624268,10545205605,10610341721,10706275505,10836514119,11226551556,11587708108,11966269637,12150412848,16134196043,16809613883,18829096237,18899271839,18926524283,18954349029,18982758094,19011763749,19041378523,19071615208,19102486862,19134006822,19166188701,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399,19199046399
New Equity ($),299223583,937287197,1044491301,1660800533,2314823401,2766922625,2985157874,3282915269,4209733106,4416602874,4624487726,4678186925,5565711565,5671538016,5708119794,5743377974,5795306931,5865805093,6076932357,6272426399,6477341755,6577018474,8733440318,9099043995,10192189793,10230175847,10244927594,10259989129,10275366956,10291067717,10307098195,10323465312,10340176139,10357237893,10374657944,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816,10392443816
New Debt ($),253563380,794261292,885106521,1407369674,1961591528,2344702398,2529635908,2781956833,3567346344,3742648695,3918811232,3964316170,4716408452,4806086252,4837085811,4866963747,4910968574,4970709027,5149619199,5315281709,5488927882,5573394373,74

### 10. Retired Plants 

In [131]:
retired_plants_df = process_retired_plants(run_variables_dict, scenario_financials_tables, ongoing_capex_df, existing_plant_NPV_EOY, financial_scalars_inputs)
# add in WACC to retired plants df as calculated in the capital charge section
retired_plants_df.loc['Return on %'] = capital_charge_df.loc['Return on (WACC)'].to_dict()

#retired_plants_df

### 11. Revenue Requirement


In [132]:
# Book Depreciation
book_depreciation = (rate_base_df.loc['Depreciation - New'] 
                    + rate_base_df.loc['Depreciation - Existing'] 
                    - total_existing_plant_summary.loc['Depreciation "Credit Back"'].fillna(0))
revenue_requirement_df = book_depreciation.to_frame().T
revenue_requirement_df.rename(index={revenue_requirement_df.index[0]: 'Book Deprecitation'}, inplace=True)

# Total O&M
total_gen_o_m = pd.Series(O_and_M_summary.loc['Total O&M Costs'], index=revenue_requirement_df.columns)
revenue_requirement_df.loc['Total Generation O&M'] = total_gen_o_m

# Capital Charge (- return on retired assets if not allowed reutrn)
capital_charge = pd.Series(capital_charge_df.loc['Return on Ratebase'] - retired_plants_df.loc['Earn Return on $'], index=revenue_requirement_df.columns)
revenue_requirement_df.loc['Capital Charge'] = capital_charge

# Taxes
income_tax_rate = financial_scalars_inputs.loc['Income Tax Rate', 'Value']
taxes = (
    (capital_charge_df.loc['ROE'] / (1 - income_tax_rate)) * income_tax_rate 
    - (retired_plants_df.loc['Income Tax'] + retired_plants_df.loc['Property Tax'])
    - total_grossed_up_ptc.loc['Grossed Up PTC'].reindex(capital_charge_df.loc['ROE'].index, fill_value=0)
    - ITC.loc['Total Grossed Up IRA ITC Benefit'].reindex(capital_charge_df.loc['ROE'].index, fill_value=0)
)
revenue_requirement_df.loc['Taxes'] = taxes

# License Fee
revenue_requirement_df.loc['License Fee'] = revenue_requirement_df.sum() * financial_scalars_inputs.loc['License Fee', 'Value']

# Total Revenue Requirement 
revenue_requirement_df.loc['Total Revenue Requirement'] = revenue_requirement_df.sum()
revenue_requirement_df = revenue_requirement_df.replace(np.nan, None)
revenue_requirement_df.style.format(precision=0)

,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081
Book Deprecitation,None,169412035,205002368,207815064,258885898,307505819,335834648,349679993,368385419,425888406,439048462,452341943,454021931,509112368,513790977,504948246,506590717,505065749,485482177,498933378,511350685,508952820,513418094,610352699,568445655,620552364,620634909,619953197,619828277,619060784,609546875,592188771,554117137,548558248,533851530,523697085,497265786,482506945,464028467,406891417,394210018,381932278,378778770,324332214,318206075,316661381,315469806,314303517,310205309,297420837,285001918,272012918,None,None,None,None,None,None,None,None,None
Total Generation O&M,None,413693636,385554966,320583869,332180918,374898402,398425054,427956521,433187195,429466463,415350351,416142803,433348932,469677497,478973446,491737064,508651532,530623764,557650224,580343081,606928004,643818660,661952787,620560839,619340944,576329668,182951993,186793985,190716659,194721708,198810864,202985892,207248596,211600817,216044434,220581367,225213576,229943061,234771865,239702074,244735818,249875270,255122651,260480226,265950311,271535268,277237508,283059496,289003745,295072824,301269353,307596010,None,None,None,None,None,None,None,None,None
Capital Charge,None,216634517,247383923,273422774,298898685,359325734,404920121,419063127,421076751,466871215,503015313,490442126,469313481,493304388,519549603,490993906,460070928,431266644,405977116,392139800,385610807,376950306,361055950,467885372,582492900,621449884,637360679,586399329,541373004,501919222,466479569,434449669,405109772,377072372,384990892,393075700,401330290,409758226,418363149,427148775,436118899,445277396,454628221,464175414,473923098,483875483,494036868,504411642,515004287,525819377,536861584,548135677,None,None,None,None,None,None,None,None,None
Taxes,None,1099052985,13754305,-35779215,-72891556,-107320022,-121436040,-131948397,-113598782,-116614923,-133284139,-167055603,-86017208,-162545201,-182736644,-168159687,-146755309,-125457783,-113705522,-85357535,-53333632,-26911094,-32611905,14996650,163626046,174569317,179038762,164723387,152075200,140992376,131037147,122039739,113797971,105922083,108146447,110417522,112736290,115103752,117520931,119988871,122508637,125081318,127708026,130389894,133128082,135923772,138778171,141692513,144668056,147706085,150807912,153974879,None,None,None,None,None,None,None,None,None
License Fee,0,60571502,27169088,24436756,26064659,29807677,32466027,33965565,35378714,38458996,39049747,38020693,40534282,41774615,42413519,42092673,42380996,42793798,42599387,44215273,46272732,47939661,47971696,54670078,61691587,63573549,51677564,49696050,47977381,46468541,44847395,43118084,40840724,39656597,39652762,39803916,39445816,39470252,39386433,38080023,38202591,38349104,38797982,37622150,37999521,38535069,39094163,39666603,40158317,40386010,40638710,40886852,0,0,0,0,0,0,0,0,0
Total Revenue Requirement,0,1959364675,878864651,790479248,843138605,964217609,1050209809,1098716808,1144429297,1244070157,1263179733,1229891962,1311201418,1351323668,1371990901,1361612202,1370938863,1384292172,1378003383,1430273996,1496828596,1550750352,1551786622,1768465638,1995597132,2056474782,1671663907,1607565948,1551970520,1503162632,1450721850,1394782155,1321114201,1282810117,1282686064,1287575591,1275991757,1276782237,1274070845,1231811160,1235775963,1240515366,1255035650,1216999899,1229207087,1246530973,1264616516,1283133770,1299039713,1306405132,1314579478,1322606335,0,0,0,0,0,0,0,0,0


In [133]:
# NPV RR

# 1. Net Present Value of All Costs (2023-2047)
discount_rate = financial_scalars_inputs.loc['After-Tax WACC', 'Value']
npv = npf.npv(discount_rate, [0] + list(revenue_requirement_df.loc['Total Revenue Requirement', 2023:2047]))
print("NPV RR for net present value of all costs 2023-2047 is ${:,.0f}".format(npv))

# 2. Long-Term NPVRR (2023-2057)
npv_long_term = npf.npv(discount_rate, [0] + list(revenue_requirement_df.loc['Total Revenue Requirement', 2023:2057]))
print("Long Term NPV RR for net present value of all costs 2023-2057 is ${:,.0f}".format(npv_long_term))

# 3. End Effects NPVRR (2023-2072)
npv_end_effects = npf.npv(discount_rate, [0] + list(revenue_requirement_df.loc['Total Revenue Requirement', 2023:2072]))
print("End Effects NPV RR for net present value of all costs 2023-2072 is ${:,.0f}".format(npv_end_effects))


## Make a df to store these

npv_df = {
    'Net Present Value of All Costs (2023-2047)': [npv],
    'Long-Term NPVRR (2023-2057)': [npv_long_term],
    'End Effects NPVRR (2023-2072)': [npv_end_effects]
}

npv_df = pd.DataFrame(npv_df)

npv_df_styled = npv_df.copy().T
#npv_df_styled = npv_df_styled.applymap(lambda x: '${:,.0f}'.format(x) if pd.notna(x) and np.issubdtype(type(x), np.number) else x)
npv_df_styled = npv_df_styled.applymap(lambda x: '${:,.0f}'.format(x) if x is not None else None)
npv_df_styled

NPV RR for net present value of all costs 2023-2047 is $14,447,451,181
Long Term NPV RR for net present value of all costs 2023-2057 is $16,428,982,061
End Effects NPV RR for net present value of all costs 2023-2072 is $17,627,481,159


,0
Net Present Value of All Costs (2023-2047),"$14,447,451,181"
Long-Term NPVRR (2023-2057),"$16,428,982,061"
End Effects NPVRR (2023-2072),"$17,627,481,159"


### 12. Make Excel Output

In [134]:
# Create Excel workbook

wb = Workbook()
worksheet= wb.active

#### 1. Details about current run
    
# Add header row
header_row = ['Model Run Variables', 'Value']
add_header_row(worksheet, header_row)

# Add data to worksheet
run_vars_df = pd.DataFrame.from_dict(run_variables_dict, orient='index', columns=['Value']).reset_index()
add_data_to_worksheet(worksheet, run_vars_df, 
                      formatting_type='Bold Text')

#### 2. Revenue Requirement Calc

# Add header row
header_row = ['Revenue Requirement Calculation']
add_header_row(worksheet, header_row)

revenue_requirement_df_styled = style_dataframe_with_currency(revenue_requirement_df).reset_index().rename(columns={'index': 'Calculation Components'})
add_data_to_worksheet(worksheet, 
                      revenue_requirement_df_styled,
                      use_cols_as_header = True)


#### 3. Total NPV

# Add header row
header_row = ['Net Present Value of Revenue Requirement (NPV RR)']
add_header_row(worksheet, header_row, color="FAE650")

add_data_to_worksheet(worksheet, 
                      npv_df_styled.reset_index(),
                      use_cols_as_header = False,
                      formatting_type = "Bold Text and Green Money",
                      bold_last_row=True)



#### 4. O&M
header_row = ['O&M Summary']
add_header_row(worksheet, header_row)


VOM_portfolio_cost_df_styled = style_dataframe_with_currency(VOM_portfolio_cost_df).reset_index().rename(columns={'index': 'Variable O&M'})
FOM_yearly_general_df_styled = style_dataframe_with_currency(FOM_yearly_general_df).reset_index().rename(columns={'index': 'New Unit FOM'})
FOM_portfolio_cost_df_styled = style_dataframe_with_currency(FOM_portfolio_cost_df).reset_index().rename(columns={'index': 'Other Fixed Costs'})
AS_RT_portfolio_cost_df = AS_RT_portfolio_cost_df.rename(index={"New Unit Subhourly / Ancillary Revenue": "Total New Unit Subhourly / Ancillary Revenue"})
AS_RT_portfolio_cost_df_styled = style_dataframe_with_currency(AS_RT_portfolio_cost_df).reset_index().rename(columns={'index': 'New Unit Subhourly / Ancillary Revenue'})

add_data_to_worksheet(worksheet, 
                      VOM_portfolio_cost_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money")
add_data_to_worksheet(worksheet, 
                      FOM_yearly_general_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money")
add_data_to_worksheet(worksheet, 
                      FOM_portfolio_cost_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_first_row = True)
add_data_to_worksheet(worksheet, 
                      AS_RT_portfolio_cost_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money")

#### 5. Rate Base
header_row = ['Rate Base Calculation']
add_header_row(worksheet, header_row)

rate_base_df_styled = style_dataframe_with_currency(rate_base_df).reset_index().rename(columns={'index': 'Rate Base Components'})
add_data_to_worksheet(worksheet, 
                      rate_base_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_first_row = False)

subhead_color = "e0e4f4"
header_row = ['New CapEx']
add_header_row(worksheet, header_row, color=subhead_color)

new_capex_df_styled = style_dataframe_with_currency(new_capex_df).reset_index().rename(columns={0: 'Resource'})
add_data_to_worksheet(worksheet, 
                      new_capex_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_first_row = False)

header_row = ['New CapEx with Accumulated AFUDC']
add_header_row(worksheet, header_row, color=subhead_color)

new_capex_with_accumulated_AFUDC_df_styled = style_dataframe_with_currency(new_capex_with_accumulated_AFUDC).reset_index().rename(columns={0: 'Resource'})
add_data_to_worksheet(worksheet, 
                      new_capex_with_accumulated_AFUDC_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_first_row = False)

header_row = ['Ongoing CapEx']
add_header_row(worksheet, header_row, color=subhead_color)

ongoing_capex_df_styled = style_dataframe_with_currency(ongoing_capex_df).reset_index().rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      ongoing_capex_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_first_row = False)

#### 6. Existing Plant
header_row = ['Existing Plant']
add_header_row(worksheet, header_row)

header_row = ['Existing Plant NBV BOY']
add_header_row(worksheet, header_row, color=subhead_color)

existing_plant_NPV_BOY_styled = style_dataframe_with_currency(existing_plant_NPV_BOY).reset_index()#.rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      existing_plant_NPV_BOY_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['Existing Plant NBV EOY']
add_header_row(worksheet, header_row, color=subhead_color)

existing_plant_NPV_EOY_styled = style_dataframe_with_currency(existing_plant_NPV_EOY).reset_index()#.rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      existing_plant_NPV_EOY_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['Existing Plant Depreciation']
add_header_row(worksheet, header_row, color=subhead_color)

existing_plant_depreciation_styled = style_dataframe_with_currency(existing_plant_depreciation).reset_index().rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      existing_plant_depreciation_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

existing_plant_other_items = total_existing_plant_summary.loc[["Additions to Existing Book", "Depreciation \"Credit Back\""]]

existing_plant_other_items_styled = style_dataframe_with_currency(existing_plant_other_items).reset_index().rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      existing_plant_other_items_styled,
                      use_cols_as_header = False,
                      formatting_type = "Normal Text and Blue Money")

#### 7. Retired Plants

header_row = ['Retired Plants']
add_header_row(worksheet, header_row)

retired_plants_df_styled = style_dataframe_with_currency(retired_plants_df).reset_index()#.rename(columns={'index': 'Resource'})

add_data_to_worksheet(worksheet, 
                      retired_plants_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

#### 7. AFUDC Calculation

header_row = ['AFUDC Calculation']
add_header_row(worksheet, header_row)

header_row = ['AFUDC']
add_header_row(worksheet, header_row, color=subhead_color)

AFUDC_schedule_df_styled = style_dataframe_with_currency(AFUDC_schedule_df).reset_index().rename(columns={0: 'Resource'})
add_data_to_worksheet(worksheet, 
                      AFUDC_schedule_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['AFUDC with Rate']
add_header_row(worksheet, header_row, color=subhead_color)

AFUDC_with_rate_df_styled = style_dataframe_with_currency(AFUDC_with_rate_df).reset_index().rename(columns={0: 'Resource'})
add_data_to_worksheet(worksheet, 
                      AFUDC_with_rate_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['Accumulated AFUDC']
add_header_row(worksheet, header_row, color=subhead_color)

AFUDC_accumulated_df_styled = style_dataframe_with_currency(AFUDC_accumulated_df).reset_index().rename(columns={0: 'Resource'})
add_data_to_worksheet(worksheet, 
                      AFUDC_accumulated_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

#### 7. Deferred Tax Calculation

header_row = ['Deferred Tax Calculations']
add_header_row(worksheet, header_row)

header_row = ['Deferred Tax Calculations - Blended - Weight by Fed/State Tax Rate']
add_header_row(worksheet, header_row, color=subhead_color)

deferred_tax_blended_df_styled = style_dataframe_with_currency(deferred_tax_blended_df).reset_index().rename(columns={'index': 'Component'})
add_data_to_worksheet(worksheet, 
                      deferred_tax_blended_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['Deferred Tax Calculations - Federal Only']
add_header_row(worksheet, header_row, color=subhead_color)

deferred_tax_federal_df_styled = style_dataframe_with_currency(deferred_tax_federal_df).reset_index().rename(columns={'index': 'Component'})
add_data_to_worksheet(worksheet, 
                      deferred_tax_federal_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['Deferred Tax Calculations - State Only']
add_header_row(worksheet, header_row, color=subhead_color)

deferred_tax_state_df_styled = style_dataframe_with_currency(deferred_tax_state_df).reset_index().rename(columns={'index': 'Component'})
add_data_to_worksheet(worksheet, 
                      deferred_tax_state_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

#### 8. Capital Charge Calculation

header_row = ['Capital Charge Calculation']
add_header_row(worksheet, header_row)

capital_charge_df_styled = style_dataframe_with_currency(capital_charge_df).reset_index().rename(columns={'index': 'Component'})
add_data_to_worksheet(worksheet, 
                      capital_charge_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

#### 9. Tax Credit Calc

header_row = ['Tax Credit Calculation']
add_header_row(worksheet, header_row)

header_row = ['PTC']
add_header_row(worksheet, header_row, color=subhead_color)

PTC_styled = style_dataframe_with_currency(PTC_df).reset_index().rename(columns={'index': 'PTC Prices'})
add_data_to_worksheet(worksheet, 
                      PTC_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['Generation (MWh, kg H2, ton CO2)']
add_header_row(worksheet, header_row, color=subhead_color)

generation_df_styled = style_dataframe_with_currency(generation_df).reset_index().rename(columns={'index': 'Resource'})
add_data_to_worksheet(worksheet, 
                      generation_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = False)

header_row = ['Old Tax Policy PTC Generated']
add_header_row(worksheet, header_row, color=subhead_color)

old_tax_policy_PTC_generated_styled = style_dataframe_with_currency(old_tax_policy_PTC_generated).reset_index().rename(columns={'index': 'PTC'})
add_data_to_worksheet(worksheet, 
                      old_tax_policy_PTC_generated_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['IRA PTC']
add_header_row(worksheet, header_row, color=subhead_color)

IRA_PTC_df_styled = style_dataframe_with_currency(IRA_PTC_df).reset_index().rename(columns={'index': 'PTC'})
add_data_to_worksheet(worksheet, 
                      IRA_PTC_df_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['Total Grossed Up PTC Payment']
add_header_row(worksheet, header_row, color=subhead_color)

total_grossed_up_ptc_styled = style_dataframe_with_currency(total_grossed_up_ptc).reset_index().rename(columns={'index': 'PTC'})
add_data_to_worksheet(worksheet, 
                      total_grossed_up_ptc_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['NOL?']
add_header_row(worksheet, header_row, color=subhead_color)

NOL_styled = style_dataframe_with_currency(NOL).reset_index().rename(columns={'index': 'NOL?'})
add_data_to_worksheet(worksheet, 
                      NOL_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

header_row = ['ITC']
add_header_row(worksheet, header_row, color=subhead_color)

ITC_styled = style_dataframe_with_currency(ITC).reset_index().rename(columns={'index': 'NOL?'})
add_data_to_worksheet(worksheet, 
                      ITC_styled,
                      use_cols_as_header = True,
                      formatting_type = "Normal Text and Blue Money",
                      bold_last_row = True)

#### 10. Depreciation
header_row = ["Depreciation Tables"]
add_header_row(worksheet, header_row)



display_alternating_dict_dataframes_with_headers(worksheet, book_depreciation_tables_dict, tax_depreciation_tables_dict, "Depreciation Tables", subhead_color)


#### Final Formatting & Save Excel

# Set column widths
set_column_widths(worksheet)

# Save the workbook
output_filename = 'Model Results.xlsx'
#wb.save(output_filename)

In [135]:
#wb.save(output_filename)

In [137]:
# Display link to download the file
#from IPython.display import display, FileLink
#display(FileLink(output_filename))